# Chat with CSV/Structured Data

## Project Overview
This project builds a conversational interface for querying structured data (CSV files).
Unlike PDFs which are unstructured text, CSV files contain tabular data that requires
different handling techniques.

You can:
- Ask natural language questions about your data
- Get statistical insights
- Filter and analyze data conversationally
- Combine data queries with reasoning

## Use Cases
- Sales: Analyze sales data ("What were our top products last quarter?")
- HR: Query employee data ("How many employees joined in 2024?")
- Finance: Analyze financial records
- Analytics: Exploratory data analysis through conversation

## What You'll Learn
1. Working with structured data vs unstructured text
2. LangChain CSV loader and agents
3. Pandas integration with LLMs
4. Safe code execution for data queries
5. Creating data analysis chatbots

## Step 1: Environment Setup

In [1]:
# Load environment variables
from dotenv import load_dotenv
import os

load_dotenv()
print("✅ Environment loaded")
print(f"OpenAI API Key found: {'OPENAI_API_KEY' in os.environ}")

✅ Environment loaded
OpenAI API Key found: True


## Step 2: Create Sample CSV Data

Let's create a sample sales dataset for demonstration.

In [3]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

# Set random seed for reproducibility
np.random.seed(42)

# Generate sample sales data
def create_sample_sales_data():
    """
    Create a sample sales dataset with realistic data.
    """
    # Parameters
    n_records = 1000
    start_date = datetime(2024, 1, 1)
    
    # Generate data
    data = {
        'order_id': [f'ORD{i:05d}' for i in range(1, n_records + 1)],
        'date': [start_date + timedelta(days=np.random.randint(0, 365)) for _ in range(n_records)],
        'product': np.random.choice(['Laptop', 'Phone', 'Tablet', 'Headphones', 'Smartwatch'], n_records),
        'category': np.random.choice(['Electronics', 'Accessories'], n_records, p=[0.7, 0.3]),
        'quantity': np.random.randint(1, 10, n_records),
        'unit_price': np.random.uniform(50, 1500, n_records).round(2),
        'customer_type': np.random.choice(['New', 'Returning'], n_records, p=[0.3, 0.7]),
        'region': np.random.choice(['North', 'South', 'East', 'West'], n_records),
        'payment_method': np.random.choice(['Credit Card', 'Debit Card', 'PayPal'], n_records)
    }
    
    df = pd.DataFrame(data)
    
    # Calculate total_amount
    df['total_amount'] = (df['quantity'] * df['unit_price']).round(2)
    
    # Sort by date
    df = df.sort_values('date').reset_index(drop=True)
    
    return df

# Create and save the dataset
sales_df = create_sample_sales_data()
csv_path = "sample_sales_data.csv"
sales_df.to_csv(csv_path, index=False)

print(f"✅ Created sample sales dataset: {csv_path}")
print(f"   Records: {len(sales_df)}")
print(f"   Columns: {', '.join(sales_df.columns)}")
print(f"\nFirst 5 rows:")
sales_df.head()

✅ Created sample sales dataset: sample_sales_data.csv
   Records: 1000
   Columns: order_id, date, product, category, quantity, unit_price, customer_type, region, payment_method, total_amount

First 5 rows:


,order_id,date,product,category,quantity,unit_price,customer_type,region,payment_method,total_amount
0,ORD00849,2024-01-01,Headphones,Electronics,5,839.92,New,North,PayPal,4199.60
1,ORD00678,2024-01-01,Phone,Electronics,6,237.30,Returning,West,Credit Card,1423.80
2,ORD00649,2024-01-01,Smartwatch,Electronics,9,254.61,Returning,South,PayPal,2291.49
3,ORD00756,2024-01-01,Headphones,Accessories,8,168.42,New,East,Credit Card,1347.36
4,ORD00992,2024-01-01,Laptop,Electronics,3,956.73,New,North,PayPal,2870.19


## Step 3: Basic Data Exploration

In [4]:
# Basic statistics
print("Dataset Info:")
print("="*80)
print(f"Total Records: {len(sales_df)}")
print(f"Date Range: {sales_df['date'].min()} to {sales_df['date'].max()}") # setting min to max because dates are generated within a year
print(f"Total Revenue: ${sales_df['total_amount'].sum():,.2f}") # total revenue
print(f"Average Order Value: ${sales_df['total_amount'].mean():,.2f}")
print(f"\nProducts: {', '.join(sales_df['product'].unique())}")
print(f"Regions: {', '.join(sales_df['region'].unique())}")

Dataset Info:
Total Records: 1000
Date Range: 2024-01-01 00:00:00 to 2024-12-30 00:00:00
Total Revenue: $3,863,053.77
Average Order Value: $3,863.05

Products: Headphones, Phone, Smartwatch, Laptop, Tablet
Regions: North, West, South, East


## Step 4: Import LangChain Libraries

In [8]:
# LLM and agents
from langchain_openai import ChatOpenAI
#from langchain_experimental.agents import create_pandas_dataframe_agent
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent

# Agent types
from langchain_classic.agents.agent_types import AgentType

print("✅ LangChain libraries imported")

✅ LangChain libraries imported


## Step 5: Initialize LLM

In [9]:
# Initialize OpenAI LLM
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0,  # Zero temperature for precise data queries
    api_key=os.environ["OPENAI_API_KEY"]
)

print("✅ LLM initialized")

✅ LLM initialized


## Step 6: Create Pandas Dataframe Agent

This agent can:
- Understand natural language questions
- Generate pandas code to query the dataframe
- Execute the code safely
- Return natural language answers

In [12]:
# !pip install tabulate

In [13]:
# Create pandas dataframe agent
agent = create_pandas_dataframe_agent(
    llm=llm,
    df=sales_df,
    agent_type=AgentType.OPENAI_FUNCTIONS,  # Use function calling for better reliability
    verbose=True,  # Show intermediate steps
    allow_dangerous_code=True  # Required for pandas agent (executes generated code)
)

print("✅ Pandas DataFrame Agent created")
print("   Agent can now answer questions about the sales data!")

✅ Pandas DataFrame Agent created
   Agent can now answer questions about the sales data!


## Step 7: Query the Data with Natural Language

### Simple Statistics Questions

In [14]:
# Question 1: Total revenue
question1 = "What is the total revenue from all sales?"
print(f"Question: {question1}\n")
response1 = agent.invoke(question1)
print(f"\nAnswer: {response1['output']}")
print("\n" + "="*80)

Question: What is the total revenue from all sales?



> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': "total_revenue = df['total_amount'].sum()"}`



Invoking: `python_repl_ast` with `{'query': 'total_revenue'}`


3863053.77The total revenue from all sales is $3,863,053.77.

> Finished chain.

Answer: The total revenue from all sales is $3,863,053.77.



In [15]:
# Question 2: Best selling product
question2 = "Which product generated the most revenue?"
print(f"Question: {question2}\n")
response2 = agent.invoke(question2)
print(f"\nAnswer: {response2['output']}")
print("\n" + "="*80)

Question: Which product generated the most revenue?



> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': "df.groupby('product')['total_amount'].sum().idxmax()"}`


LaptopThe product that generated the most revenue is the **Laptop**.

> Finished chain.

Answer: The product that generated the most revenue is the **Laptop**.



In [16]:
# Question 3: Average order value by region
question3 = "What is the average order value for each region?"
print(f"Question: {question3}\n")
response3 = agent.invoke(question3)
print(f"\nAnswer: {response3['output']}")
print("\n" + "="*80)

Question: What is the average order value for each region?



> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': "df.groupby('region')['total_amount'].mean()"}`


region
East     4082.158080
North    3795.232439
South    3573.305314
West     4036.572232
Name: total_amount, dtype: float64The average order value for each region is as follows:

- East: 4082.16
- North: 3795.23
- South: 3573.31
- West: 4036.57

> Finished chain.

Answer: The average order value for each region is as follows:

- East: 4082.16
- North: 3795.23
- South: 3573.31
- West: 4036.57



### Complex Analytical Questions

In [17]:
# Question 4: Customer behavior analysis
question4 = "How many new customers vs returning customers made purchases, and what's the difference in their average spending?"
print(f"Question: {question4}\n")
response4 = agent.invoke(question4)
print(f"\nAnswer: {response4['output']}")
print("\n" + "="*80)

Question: How many new customers vs returning customers made purchases, and what's the difference in their average spending?



> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': "new_customers = df[df['customer_type'] == 'New']\nreturning_customers = df[df['customer_type'] == 'Returning']\n\nnew_customers_count = new_customers['customer_type'].count()\nreturning_customers_count = returning_customers['customer_type'].count()\n\nnew_customers_avg_spending = new_customers['total_amount'].mean()\nreturning_customers_avg_spending = returning_customers['total_amount'].mean()\n\navg_spending_difference = new_customers_avg_spending - returning_customers_avg_spending\n\n(new_customers_count, returning_customers_count, new_customers_avg_spending, returning_customers_avg_spending, avg_spending_difference)"}`


(np.int64(309), np.int64(691), np.float64(3866.462394822006), np.float64(3861.5295079594794), np.float64(4.93288686252663))In the dataset:

- **New customer

In [ ]:
# Question 5: Seasonal trends
question5 = "Which month had the highest sales revenue in 2024?"
print(f"Question: {question5}\n")
response5 = agent.invoke(question5)
print(f"\nAnswer: {response5['output']}")
print("\n" + "="*80)

In [ ]:
# Question 6: Multi-step reasoning
question6 = "What percentage of total revenue came from Laptops, and how does that compare to the average across all products?"
print(f"Question: {question6}\n")
response6 = agent.invoke(question6)
print(f"\nAnswer: {response6['output']}")
print("\n" + "="*80)

## Step 8: Interactive Query Function

In [18]:
def query_sales_data(question: str, show_steps: bool = False):
    """
    Query the sales data with natural language.
    
    Args:
        question: Natural language question about the data
        show_steps: Whether to show intermediate reasoning steps
    
    Returns:
        Answer string
    """
    # Recreate agent with verbose setting
    temp_agent = create_pandas_dataframe_agent(
        llm=llm,
        df=sales_df,
        agent_type=AgentType.OPENAI_FUNCTIONS,
        verbose=show_steps,
        allow_dangerous_code=True
    )
    
    print(f"\n{'='*80}")
    print(f"QUESTION: {question}")
    print(f"{'='*80}\n")
    
    result = temp_agent.invoke(question)
    
    print(f"\nANSWER: {result['output']}")
    print(f"\n{'='*80}\n")
    
    return result['output']

# Example usage
query_sales_data(
    "What are the top 3 products by quantity sold?",
    show_steps=True
)


QUESTION: What are the top 3 products by quantity sold?



> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': "top_products = df.groupby('product')['quantity'].sum().nlargest(3) \ntop_products"}`


product
Laptop        1052
Smartwatch    1013
Phone         1005
Name: quantity, dtype: int32The top 3 products by quantity sold are:

1. Laptop: 1052 units
2. Smartwatch: 1013 units
3. Phone: 1005 units

> Finished chain.

ANSWER: The top 3 products by quantity sold are:

1. Laptop: 1052 units
2. Smartwatch: 1013 units
3. Phone: 1005 units




'The top 3 products by quantity sold are:\n\n1. Laptop: 1052 units\n2. Smartwatch: 1013 units\n3. Phone: 1005 units'

## Step 9: Try Your Own Questions

Now experiment with your own questions!

In [21]:
# Custom questions 
query_sales_data("Which payment method is most popular in the West region?")


QUESTION: Which payment method is most popular in the West region?


ANSWER: The most popular payment method in the West region is Debit Card.




'The most popular payment method in the West region is Debit Card.'

In [20]:
# Another custom question
query_sales_data("Show me the correlation between quantity and total amount")


QUESTION: Show me the correlation between quantity and total amount


ANSWER: The correlation between quantity and total amount is approximately 0.664. This indicates a moderate positive correlation, suggesting that as the quantity increases, the total amount tends to increase as well.




'The correlation between quantity and total amount is approximately 0.664. This indicates a moderate positive correlation, suggesting that as the quantity increases, the total amount tends to increase as well.'

## Step 10: Working with Your Own CSV Files

Load and query your own CSV data.

In [ ]:
def create_csv_chat_agent(csv_path: str):
    """
    Create a chat agent for any CSV file.
    
    Args:
        csv_path: Path to CSV file
    
    Returns:
        Configured agent
    """
    # Load CSV
    df = pd.read_csv(csv_path)
    
    print(f"✅ Loaded CSV: {csv_path}")
    print(f"   Rows: {len(df)}")
    print(f"   Columns: {', '.join(df.columns)}")
    print(f"\nFirst 3 rows:")
    print(df.head(3))
    print()
    
    # Create agent
    agent = create_pandas_dataframe_agent(
        llm=llm,
        df=df,
        agent_type=AgentType.OPENAI_FUNCTIONS,
        verbose=False,
        allow_dangerous_code=True
    )
    
    return agent, df

# Example: Load your own CSV
my_agent, my_df = create_csv_chat_agent("path/to/your/data.csv")
my_agent.invoke("Question here")

print("✅ Function ready to load your CSV files")

✅ Function ready to load your CSV files


## Step 11: Advanced - Combining CSV Data with External Knowledge

In [23]:
# Question that requires reasoning beyond the data
question_reasoning = """
Based on the sales data, what insights can you provide about customer behavior?
Consider patterns in customer types, regions, and payment methods.
Provide 3 actionable business recommendations.
"""

print(f"Question: {question_reasoning}\n")
response_reasoning = agent.invoke(question_reasoning)
print(f"\nAnswer:\n{response_reasoning['output']}")
print("\n" + "="*80)

Question: 
Based on the sales data, what insights can you provide about customer behavior?
Consider patterns in customer types, regions, and payment methods.
Provide 3 actionable business recommendations.




> Entering new AgentExecutor chain...
To provide insights into customer behavior based on the sales data, we can analyze the following aspects:

1. **Customer Types**:
   - **New vs. Returning Customers**: Understanding the ratio of new to returning customers can help identify customer loyalty and the effectiveness of marketing strategies aimed at attracting new customers.

2. **Regions**:
   - **Sales Distribution by Region**: Analyzing which regions have the highest sales can help identify target markets and areas that may require more marketing efforts or product availability.

3. **Payment Methods**:
   - **Preferred Payment Methods**: Identifying the most commonly used payment methods can inform the business about customer preferences and help optimize the checkout process.



## Step 12: Error Handling and Edge Cases

In [24]:
def safe_query(question: str):
    """
    Query with error handling.
    
    Args:
        question: Natural language question
    
    Returns:
        Answer or error message
    """
    try:
        result = agent.invoke(question)
        return result['output']
    except Exception as e:
        return f"Error: {str(e)}\nPlease rephrase your question or check the data."

# Test with various questions
test_questions = [
    "What is the total revenue?",
    "Show me data for a product that doesn't exist",  # Edge case
    "Calculate the growth rate" # Ambiguous question
]

for q in test_questions:
    print(f"\nQ: {q}")
    print(f"A: {safe_query(q)}")
    print("-" * 80)


Q: What is the total revenue?


> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': "total_revenue = df['total_amount'].sum()"}`



Invoking: `python_repl_ast` with `{'query': "df['total_amount'].sum()"}`


3863053.77The total revenue is $3,863,053.77.

> Finished chain.
A: The total revenue is $3,863,053.77.
--------------------------------------------------------------------------------

Q: Show me data for a product that doesn't exist


> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': "df[df['product'] == 'NonExistentProduct']"}`


Empty DataFrame
Columns: [order_id, date, product, category, quantity, unit_price, customer_type, region, payment_method, total_amount]
Index: []The query for a product that doesn't exist returned an empty dataframe, indicating that there are no records for a product named 'NonExistentProduct'.

> Finished chain.
A: The query for a product that doesn't exist returned an empty dataframe, ind

## Summary

### What You've Built:
- ✅ CSV data loading and exploration
- ✅ Natural language interface for data queries
- ✅ Pandas DataFrame Agent for automated analysis
- ✅ Complex analytical queries with multi-step reasoning
- ✅ Reusable functions for any CSV file
- ✅ Error handling for robust queries

### Key Concepts Learned:
1. **Structured vs Unstructured Data**:
   - PDFs: Unstructured text → Use embeddings + vector search
   - CSV: Structured tables → Use pandas + agents

2. **Pandas DataFrame Agent**:
   - Generates pandas code from natural language
   - Executes code safely in controlled environment
   - Returns natural language answers

3. **Agent Types**:
   - `OPENAI_FUNCTIONS`: Uses function calling (most reliable)
   - Better at multi-step reasoning
   - More structured output

4. **Safety Considerations**:
   - `allow_dangerous_code=True` required for execution
   - Agent runs generated Python code
   - Use in controlled environments only

### Structured Data Capabilities:
- ✅ Statistical analysis (sum, mean, median, etc.)
- ✅ Filtering and grouping
- ✅ Time series analysis
- ✅ Correlation and comparisons
- ✅ Multi-step reasoning
- ✅ Business insights generation

### Limitations:
- ⚠️ May struggle with very complex multi-step queries
- ⚠️ Code execution can fail on ambiguous questions
- ⚠️ Requires well-structured CSV data
- ⚠️ Performance depends on dataframe size

### Next Steps:
- Try with your own CSV files (sales, customer, financial data)
- Experiment with different question types
- Combine multiple CSV files
- Add visualization generation
- Create a full data analysis chatbot
- Integrate with SQL databases for larger datasets